# W200 Project 2

Team: Nicholas Lee, Che Guan, Tatianna Martinez, Judge Hiciano

Links:

Original DataSet: https://collegescorecard.ed.gov/data/

# Assumptions
* Dataset was filtered to only include predominantly bachelor granting
* Use data dictionary for information about the variable

# Download Data

In [1]:
!wget https://ed-public-download.app.cloud.gov/downloads/CollegeScorecard_Raw_Data_03142022.zip &> /dev/null
!echo "Downloaded Data"

Downloaded Data


In [2]:
!unzip -d ./content CollegeScorecard_Raw_Data_03142022.zip &> /dev/null
!echo "Uncomporessed Data"

Uncomporessed Data


In [3]:
# Adding the data dictionary to help with labeling information
!wget https://collegescorecard.ed.gov/assets/CollegeScorecardDataDictionary.xlsx &> /dev/null
!echo "Dictionary downloaded"

Dictionary downloaded


# Cleanup Process

## Install Python Packages

In [4]:
!pip install pandas numpy seaborn sklearn matplotlib shap &> /dev/null
!echo "Installed Python packages"

Installed Python packages


### Install packages

In [5]:
import csv
import glob
import json
import os
import sys

import yaml
import pandas as pd
import numpy as np
import seaborn as sbn
import matplotlib.pyplot as plt

import shap

from google.colab import drive
from google.colab import files

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)
pd.options.display.float_format = '{:,.2f}'.format

## Functions

In [6]:
# Deprecated since I use the data dictionary instead of the yaml file to find descriptions
from yaml import error
# Load yaml file
with open('./content/data.yaml') as file: 
  yaml_file = yaml.safe_load(file)

def description_finder(col):
  """ To find the column description within the yaml file"""
  for key in yaml_file['dictionary'].keys():
    try: 
      if yaml_file['dictionary'][key]['source'] == col: 
        return str(yaml_file['dictionary'][key]['description'])
    except KeyError: 
      return "Description Not Found"

In [7]:
def read_addyear(file): 
  """ Function to add the year from each MERGED* file to be used with merging. 
  This function will also be used to capture the necessary columns needed for our analysis"""
  df_temp = pd.read_csv(os.path.join(file))
  df_temp['ENDYEAR'] = "".join([file.split('_')[0][-4:-2], file.split('_')[1]])
  return df_temp


In [8]:
def type_finder(col) :
  """ To find the column type within the yaml file"""
  typer = {
      'integer': 'int',
      'autocomplete' : 'object',
      'float': 'float64', 
      'string':'str',
      'long':'float64'
  }
  for key in yaml_file['dictionary'].keys():
    try: 
      if yaml_file['dictionary'][key]['source'] == col: 
        return str(typer[yaml_file['dictionary'][key]['type']])
    except KeyError: 
      return 'object'

In [9]:
def type_setter(data, col): 
  """ Change the type of the column after reading in """
  data[col] = data[col].astype(type_finder(col))

In [10]:
def ref_finder(column):
  """ Find information about the variable from the data dictionary """
  if type(column) == str: 
    return df_dict[df_dict['variable_name'] == (column)]
  elif type(column) == list:
    return df_dict[df_dict['variable_name'].isin(column)]
  else: 
    return "Please enter a variable name or list"

In [11]:
def dictionary_getter(data, column): 
  """ Create dictionary for column based on dataset dictionary """
  values = data[data['variable_name'] == column]['value']
  labels = data[data['variable_name'] == column]['label']
  return dict(zip(values,labels))

In [12]:
def column_dict_convt(data, dictionary_data, column): 
  """ Exchange values encoded as numeric with dictionary values to turn column into categorical variable.
  Provide column name as a string, dictionary as an object and data as an object."""
  dictionary = dictionary_getter(dictionary_data, column)
  data[column] = data[column].map(dictionary)
  # Sanity check on values changed
  print(data[column].value_counts(dropna = False))

## Dictionary Load and Cleanup, Merge Dataset


In [13]:
# Load data dictionary 
df_dict = pd.read_excel("./CollegeScorecardDataDictionary.xlsx", sheet_name='Institution_Data_Dictionary')

In [14]:
df_dict.columns

Index(['NAME OF DATA ELEMENT', 'dev-category', 'developer-friendly name',
       'API data type', 'INDEX', 'VARIABLE NAME', 'VALUE', 'LABEL', 'SOURCE',
       'SHOWN/USE ON SITE', 'NOTES'],
      dtype='object')

In [15]:
# Cleanup dictionary column names
print(df_dict.columns) # before columns

# Changes to the column names to make them easier to access
new_columns = list(map(lambda st: str.replace(st, ' ', "_").lower(), df_dict.columns.tolist()))
new_columns = list(map(lambda st: str.replace(st, r'/', "_").lower(), new_columns))
new_columns = list(map(lambda st: str.replace(st, '-', "_").lower(), new_columns))

print(new_columns) # after column changes

new_col_dict = dict(zip(df_dict.columns.tolist(), new_columns))
df_dict = df_dict.rename(columns = new_col_dict)

# recode CIP01CERT1, CIP01CERT2, AND CIP01ASSOC
df_dict.iloc[339, 7] = np.nan
df_dict.iloc[340, 7] = np.nan
df_dict.iloc[341, 7] = np.nan

for col in ['name_of_data_element', 'dev_category', 'developer_friendly_name', 'api_data_type', 'index', 'variable_name']: 
  # forward fill values in data dictionary to make accessing information easier
  df_dict[col] = df_dict[col].ffill()
df_dict['value'] = df_dict['value'].replace(" ", np.nan)
# df_dict['value'].value_counts(dropna = False)

Index(['NAME OF DATA ELEMENT', 'dev-category', 'developer-friendly name',
       'API data type', 'INDEX', 'VARIABLE NAME', 'VALUE', 'LABEL', 'SOURCE',
       'SHOWN/USE ON SITE', 'NOTES'],
      dtype='object')
['name_of_data_element', 'dev_category', 'developer_friendly_name', 'api_data_type', 'index', 'variable_name', 'value', 'label', 'source', 'shown_use_on_site', 'notes']


In [16]:
df_dict.iloc[339:342,:] # check to make sure the values were properly replaced. The previous values also represent the encoding for this category of variables "CIP*"

,name_of_data_element,dev_category,developer_friendly_name,api_data_type,index,variable_name,value,label,source,shown_use_on_site,notes
339,Certificate of less than one academic year in ...,academics,program.certificate_lt_1_yr.agriculture,integer,float,CIP01CERT1,0,NaN,IPEDS,NaN,Program designations at the 2-digit CIP level ...
340,Certificate of at least one but less than two ...,academics,program.certificate_lt_2_yr.agriculture,integer,float,CIP01CERT2,1,NaN,IPEDS,NaN,ibid
341,"Associate degree in Agriculture, Agriculture O...",academics,program.assoc.agriculture,integer,float,CIP01ASSOC,2,NaN,IPEDS,NaN,ibid


In [17]:
# Create Files path wildcard string
joined_files = os.path.join("./content", "MERGED*.csv")

# Create list of full paths for all MERGED*.csv files
joined_list = glob.glob(joined_files)

# Create Pandas dataframe from merged csv files
df = pd.concat(map(read_addyear, joined_list), ignore_index=True)
df.loc[df.ENDYEAR == '1900','ENDYEAR']='2000'

Columns (1,1537,1540,1542,1606,1614,1615) have mixed types.Specify dtype option on import or set low_memory=False.
Columns (1,6,1351,1408,1431,1432,1433,1445,1446,1447,1451,1452,1459,1460,1461,1465,1466,1473,1474,1475,1537,1538,1539,1540,1542,1547,1553,1554,1559,1560,1606,1610,1611,1614,1615,1616,1729,1909,1910,1911,1912,1913) have mixed types.Specify dtype option on import or set low_memory=False.
Columns (1,6,1351,1365,1377,1378,1407,1408,1411,1426,1431,1432,1433,1445,1446,1447,1461,1489,1537,1538,1539,1540,1541,1542,1547,1548,1549,1553,1554,1559,1560,1561,1562,1563,1567,1568,1571,1572,1573,1574,1606,1609,1610,1611,1613,1614,1615,1616,1636,1637,1711,1723,1724,1729,1909,1910,1911,1912,1913) have mixed types.Specify dtype option on import or set low_memory=False.
Columns (1547,1575,1576,1577,1587,1588,1729,1909,1910,1911,1912,1913) have mixed types.Specify dtype option on import or set low_memory=False.
Columns (1) have mixed types.Specify dtype option on import or set low_memory=False

In [18]:
df[['ADM_RATE','UGDS', 'UGDS_WOMEN', 'UGDS_MEN', 'UGDS_WHITE', 'UGDS_BLACK', 'UGDS_HISP', 'UGDS_ASIAN', 'UGDS_AIAN']].describe(include = 'all')

,ADM_RATE,UGDS,UGDS_WOMEN,UGDS_MEN,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN
count,"46,901.00","150,618.00","150,582.00","150,582.00","80,240.00","80,240.00","80,240.00","80,240.00","80,240.00"
mean,0.69,"2,204.51",0.65,0.35,0.46,0.17,0.15,0.03,0.01
std,0.22,"4,971.20",0.24,0.24,0.32,0.22,0.22,0.07,0.07
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.56,111.00,0.53,0.14,0.15,0.02,0.01,0.00,0.00
50%,0.72,457.50,0.63,0.36,0.50,0.08,0.06,0.01,0.00
75%,0.86,"1,970.00",0.86,0.47,0.73,0.22,0.17,0.03,0.01
max,1.00,"253,594.00",1.00,1.00,1.00,1.00,1.00,1.00,1.00


In [19]:
# Non null counts and dtypes of each column
df.info(verbose = True, show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170026 entries, 0 to 170025
Data columns (total 2990 columns):
 #     Column                            Non-Null Count   Dtype  
---    ------                            --------------   -----  
 0     UNITID                            170026 non-null  int64  
 1     OPEID                             170026 non-null  object 
 2     OPEID6                            170026 non-null  int64  
 3     INSTNM                            170026 non-null  object 
 4     CITY                              170026 non-null  object 
 5     STABBR                            170026 non-null  object 
 6     ZIP                               170026 non-null  object 
 7     ACCREDAGENCY                      6347 non-null    object 
 8     INSTURL                           6666 non-null    object 
 9     NPCURL                            6139 non-null    object 
 10    SCH_DEG                           145736 non-null  float64
 11    HCM2                  

Recode CIP. columns and categorical columns

In [20]:
# Recode cip columns to reflex categories
cip_cols = df.filter(regex = 'CIP\d{2}.').columns.tolist()

cip_value_dict = {
    0: "Program not offered" ,
    1: "Program offered",
    2: "Program offered through an exclusively distance-education program"
}

for col in cip_cols:
  df[col] = df[col].map(cip_value_dict)


# Create a list of columns to keep to change how the variable was encoded. This list is all the categorical columns
value_cols = pd.DataFrame(df_dict[~df_dict['value'].isna()]) # create data frame of all 
cat_cols = df_dict[~df_dict['value'].isna()]['variable_name'].unique().tolist()

# Convert all the object columns to have the right variable values
for col in cat_cols:
  column_dict_convt(df, value_cols, col)



Main campus        135552
Not main campus     34474
Name: MAIN, dtype: int64
Predominantly certificate-degree granting    73496
Predominantly bachelor's-degree granting     48477
Predominantly associate's-degree granting    35223
Entirely graduate-degree granting             7472
Not classified                                5358
Name: PREDDEG, dtype: int64
Certificate degree     57987
Graduate degree        45191
Associate degree       40042
Bachelor's degree      18727
Non-degree-granting     8079
Name: HIGHDEG, dtype: int64
Private for-profit    71096
Public                50657
Private nonprofit     48244
NaN                      29
Name: CONTROL, dtype: int64
California                        16886
New York                          11163
Texas                             10017
Pennsylvania                       9852
Florida                            8515
Ohio                               7943
Illinois                           6690
Missouri                           4692
Michiga

In [21]:
df[cip_cols].describe()

,CIP01CERT1,CIP01CERT2,CIP01ASSOC
count,0.00,0.00,0.00
mean,NaN,NaN,NaN
std,NaN,NaN,NaN
min,NaN,NaN,NaN
25%,NaN,NaN,NaN
50%,NaN,NaN,NaN
75%,NaN,NaN,NaN
max,NaN,NaN,NaN


In [22]:
# Example of the descriptive statistics performed on the dataset. 
# This section focuses on the admission rates, number of of students enrolled and the percentage of students that are within each of the categories
# Categories (Women, Men, White, Black, Hispanic, Asian,  American Indian/Alaska Native)
df[['ADM_RATE','UGDS', 'UGDS_WOMEN', 'UGDS_MEN', 'UGDS_WHITE', 'UGDS_BLACK', 'UGDS_HISP', 'UGDS_ASIAN', 'UGDS_AIAN']].describe(include = 'all')

,ADM_RATE,UGDS,UGDS_WOMEN,UGDS_MEN,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN
count,"46,901.00","150,618.00","150,582.00","150,582.00","80,240.00","80,240.00","80,240.00","80,240.00","80,240.00"
mean,0.69,"2,204.51",0.65,0.35,0.46,0.17,0.15,0.03,0.01
std,0.22,"4,971.20",0.24,0.24,0.32,0.22,0.22,0.07,0.07
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.56,111.00,0.53,0.14,0.15,0.02,0.01,0.00,0.00
50%,0.72,457.50,0.63,0.36,0.50,0.08,0.06,0.01,0.00
75%,0.86,"1,970.00",0.86,0.47,0.73,0.22,0.17,0.03,0.01
max,1.00,"253,594.00",1.00,1.00,1.00,1.00,1.00,1.00,1.00


In [23]:
ref_finder(['ADM_RATE','UGDS', 'UGDS_WOMEN', 'UGDS_MEN', 'UGDS_WHITE', 'UGDS_BLACK', 'UGDS_HISP', 'UGDS_ASIAN', 'UGDS_AIAN'])

,name_of_data_element,dev_category,developer_friendly_name,api_data_type,index,variable_name,value,label,source,shown_use_on_site,notes
276,Admission rate,admissions,admission_rate.overall,float,float,ADM_RATE,NaN,NaN,IPEDS,NaN,NaN
531,Enrollment of undergraduate certificate/degree...,student,size,integer,integer,UGDS,NaN,NaN,IPEDS,Yes,Not available in 2000-2001 datafile
533,Total share of enrollment of undergraduate deg...,student,demographics.race_ethnicity.white,float,integer,UGDS_WHITE,NaN,NaN,IPEDS,Yes,Not available in all datafiles due to changing...
534,Total share of enrollment of undergraduate deg...,student,demographics.race_ethnicity.black,float,integer,UGDS_BLACK,NaN,NaN,IPEDS,Yes,Not available in all datafiles due to changing...
535,Total share of enrollment of undergraduate deg...,student,demographics.race_ethnicity.hispanic,float,integer,UGDS_HISP,NaN,NaN,IPEDS,Yes,Not available in all datafiles due to changing...
536,Total share of enrollment of undergraduate deg...,student,demographics.race_ethnicity.asian,float,integer,UGDS_ASIAN,NaN,NaN,IPEDS,Yes,Not available in all datafiles due to changing...
537,Total share of enrollment of undergraduate deg...,student,demographics.race_ethnicity.aian,float,integer,UGDS_AIAN,NaN,NaN,IPEDS,Yes,Not available in all datafiles due to changing...
1986,Total share of enrollment of undergraduate deg...,student,demographics.men,float,float,UGDS_MEN,NaN,NaN,IPEDS,NaN,NaN
1987,Total share of enrollment of undergraduate deg...,student,demographics.women,float,float,UGDS_WOMEN,NaN,NaN,IPEDS,NaN,NaN


## Load and clean dataset to export

In [24]:
# FIX ZIP CODES, 5-digit plus 4 digit extension
df['ZIP'] = df['ZIP'].astype(str).apply(lambda x: x[:5])
df['ZIP'].value_counts()

# Change the type of the following columns in the dataframe. 
type_change_dict = {
    'ENDYEAR': int,
    'MD_EARN_WNE_P10': np.float64, 
    'MD_EARN_WNE_P6': np.float64, 
    'MD_EARN_WNE_P8': np.float64
}

# Remove all values in the dataset that are 'PrivacySuppressed' by setting them to NaN
df = df.replace('PrivacySuppressed', np.nan)

# Perform converstion with dictionary above. By removing the PrivacySuppressed columns the columns in the dictionary can be changed to float types
df = df.astype(type_change_dict)

# Check changes
df.info(verbose = True, show_counts = True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170026 entries, 0 to 170025
Data columns (total 2990 columns):
 #     Column                            Non-Null Count   Dtype  
---    ------                            --------------   -----  
 0     UNITID                            170026 non-null  int64  
 1     OPEID                             170026 non-null  object 
 2     OPEID6                            170026 non-null  int64  
 3     INSTNM                            170026 non-null  object 
 4     CITY                              170026 non-null  object 
 5     STABBR                            170026 non-null  object 
 6     ZIP                               170026 non-null  object 
 7     ACCREDAGENCY                      6347 non-null    object 
 8     INSTURL                           6666 non-null    object 
 9     NPCURL                            6139 non-null    object 
 10    SCH_DEG                           145736 non-null  float64
 11    HCM2                  

### Columns to Keep

#### Initial sweep to clean up dataset
* Not in use but was important to the process that was passed onto Tableau Prep to remove additional columns.

In [25]:
# # Pulling out the columns to use in our analysis 
# cols_search = ".BACH.|NPT.|PCIP(\d{2})|FIRSTGEN.|.EARN.|ENDYEAR|INSTNM|UGDS*|COST.|TUTITION.|MDCOMP_PD|SAT.|ACT." # this regular expression should capture all the columns needed
# cols_keeping = df.filter(regex=cols_search).columns.tolist()
# for i in institution_cols: 
#   cols_keeping.append(i)
# cols_keeping.append('MAIN')

# # Checking columns pulled
# cols_keeping.sort()

In [26]:
# Pulling out SAT and ACT scores for a peek, not included in the final dataset
# act_sat_search = "SAT.|ACT."
# act_sat_cols = df.filter(regex=act_sat_search).columns.tolist()
# print(act_sat_cols)

# cols_keeping_cleaned = cols_keeping + act_sat_cols
# print(cols_keeping_cleaned)

#### Columns Keeping from the Tableau Prep Cleanup
List of final colums to keep after clean up from inital sweep, and tableau exploration. 

In [27]:
cols_keeping = ['ADM_RATE', 'ADM_RATE_ALL', 'CIP01BACHL', 'CIP03BACHL', 'CIP04BACHL', 'CIP05BACHL', 'CIP09BACHL', 'CIP10BACHL', 
                'CIP11BACHL', 'CIP12BACHL', 'CIP13BACHL', 'CIP14BACHL', 'CIP15BACHL', 'CIP16BACHL', 'CIP19BACHL', 'CIP22BACHL',
                'CIP23BACHL', 'CIP24BACHL', 'CIP25BACHL', 'CIP26BACHL', 'CIP27BACHL', 'CIP29BACHL', 'CIP30BACHL', 'CIP31BACHL', 
                'CIP38BACHL', 'CIP39BACHL', 'CIP40BACHL', 'CIP41BACHL', 'CIP42BACHL', 'CIP43BACHL', 'CIP44BACHL', 'CIP45BACHL', 
                'CIP46BACHL', 'CIP47BACHL', 'CIP48BACHL', 'CIP49BACHL', 'CIP50BACHL', 'CIP51BACHL', 'CIP52BACHL', 'CIP54BACHL', 
                'CITY', 'CONTROL', 'COSTT4_A', 'ENDYEAR', 'HBCU', 'HIGHDEG', 'HSI', 'INSTNM', 'MAIN', 'MDCOST_ALL', 'MDCOST_PD', 
                'MDEARN_ALL', 'MDEARN_PD', 'MD_EARN_WNE_INC1_P10', 'MD_EARN_WNE_INC1_P6', 'MD_EARN_WNE_INC1_P8', 'MD_EARN_WNE_INC2_P10', 
                'MD_EARN_WNE_INC2_P6', 'MD_EARN_WNE_INC2_P8', 'MD_EARN_WNE_INC3_P10', 'MD_EARN_WNE_INC3_P6', 'MD_EARN_WNE_INC3_P8', 
                'MD_EARN_WNE_INDEP0_P10', 'MD_EARN_WNE_INDEP0_P6', 'MD_EARN_WNE_INDEP0_P8', 'MD_EARN_WNE_INDEP1_P10', 'MD_EARN_WNE_INDEP1_P6', 
                'MD_EARN_WNE_INDEP1_P8', 'MD_EARN_WNE_MALE0_P10', 'MD_EARN_WNE_MALE0_P6', 'MD_EARN_WNE_MALE0_P8', 'MD_EARN_WNE_MALE1_P10', 
                'MD_EARN_WNE_MALE1_P6', 'MD_EARN_WNE_MALE1_P8', 'MD_EARN_WNE_P10', 'MD_EARN_WNE_P6', 'MD_EARN_WNE_P8', 'MENONLY', 'NANTI', 
                'NPT41_PRIV', 'NPT41_PUB', 'NPT42_PRIV', 'NPT42_PUB', 'NPT43_PRIV', 'NPT43_PUB', 'NPT44_PRIV', 'NPT44_PUB', 'NPT45_PRIV', 
                'NPT45_PUB', 'NPT4_048_PRIV', 'NPT4_048_PUB', 'NPT4_3075_PRIV', 'NPT4_3075_PUB', 'NPT4_75UP_PRIV', 'NPT4_75UP_PUB', 
                'NPT4_PRIV', 'NPT4_PUB', 'NUMBRANCH', 'PBI', 'PCIP01', 'PCIP03', 'PCIP04', 'PCIP05', 'PCIP09', 'PCIP10', 'PCIP11', 'PCIP12', 
                'PCIP13', 'PCIP14', 'PCIP15', 'PCIP16', 'PCIP19', 'PCIP22', 'PCIP23', 'PCIP24', 'PCIP25', 'PCIP26', 'PCIP27', 'PCIP29', 
                'PCIP30', 'PCIP31', 'PCIP38', 'PCIP39', 'PCIP40', 'PCIP41', 'PCIP42', 'PCIP43', 'PCIP44', 'PCIP45', 'PCIP46', 'PCIP47', 
                'PCIP48', 'PCIP49', 'PCIP50', 'PCIP51', 'PCIP52', 'PCIP54', 'PCT25_EARN_WNE_P6', 'PCT25_EARN_WNE_P8', 'PCT25_EARN_WNE_P10', 
                'PCT75_EARN_WNE_P6', 'PCT75_EARN_WNE_P8', 'PCT75_EARN_WNE_P10', 'PREDDEG', 'REGION', 'RELAFFIL', 'SCH_DEG', 'STATE', 'TRIBAL', 
                'UGDS', 'UGDS_2MOR', 'UGDS_AIAN', 'UGDS_ASIAN', 'UGDS_BLACK', 'UGDS_HISP', 'UGDS_MEN', 'UGDS_NHPI', 'UGDS_NRA', 'UGDS_UNKN', 
                'UGDS_WHITE', 'UGDS_WOMEN', 'WOMENONLY', 'ZIP', 'MDCOMP_PD', 'STABBR', 'ST_FIPS', 'REGION', 'CCBASIC', 'CCUGPROF', 'UNITID', 'CCSIZSET',
                "OVERALL_YR2_N", "OVERALL_YR3_N", "OVERALL_YR4_N", "OVERALL_YR6_N", "OVERALL_YR8_N"]


In [28]:
cols_keeping.sort()
print(cols_keeping)

['ADM_RATE', 'ADM_RATE_ALL', 'CCBASIC', 'CCSIZSET', 'CCUGPROF', 'CIP01BACHL', 'CIP03BACHL', 'CIP04BACHL', 'CIP05BACHL', 'CIP09BACHL', 'CIP10BACHL', 'CIP11BACHL', 'CIP12BACHL', 'CIP13BACHL', 'CIP14BACHL', 'CIP15BACHL', 'CIP16BACHL', 'CIP19BACHL', 'CIP22BACHL', 'CIP23BACHL', 'CIP24BACHL', 'CIP25BACHL', 'CIP26BACHL', 'CIP27BACHL', 'CIP29BACHL', 'CIP30BACHL', 'CIP31BACHL', 'CIP38BACHL', 'CIP39BACHL', 'CIP40BACHL', 'CIP41BACHL', 'CIP42BACHL', 'CIP43BACHL', 'CIP44BACHL', 'CIP45BACHL', 'CIP46BACHL', 'CIP47BACHL', 'CIP48BACHL', 'CIP49BACHL', 'CIP50BACHL', 'CIP51BACHL', 'CIP52BACHL', 'CIP54BACHL', 'CITY', 'CONTROL', 'COSTT4_A', 'ENDYEAR', 'HBCU', 'HIGHDEG', 'HSI', 'INSTNM', 'MAIN', 'MDCOMP_PD', 'MDCOST_ALL', 'MDCOST_PD', 'MDEARN_ALL', 'MDEARN_PD', 'MD_EARN_WNE_INC1_P10', 'MD_EARN_WNE_INC1_P6', 'MD_EARN_WNE_INC1_P8', 'MD_EARN_WNE_INC2_P10', 'MD_EARN_WNE_INC2_P6', 'MD_EARN_WNE_INC2_P8', 'MD_EARN_WNE_INC3_P10', 'MD_EARN_WNE_INC3_P6', 'MD_EARN_WNE_INC3_P8', 'MD_EARN_WNE_INDEP0_P10', 'MD_EARN_WNE_IN

In [29]:
len(cols_keeping)

176

#### Refine dataset with columns keeping
* Use the columns to reduce the size of the data frame.  

In [30]:
print("Original shape: ", df.shape)

# Columns to keep
df = df.filter(cols_keeping)
df = df[df['PREDDEG'] == "Predominantly bachelor's-degree granting"]

print("Shape after column removal: ", df.shape)
print("-" * 50)


Original shape:  (170026, 2990)
Shape after column removal:  (48477, 175)
--------------------------------------------------


In [31]:
# Create a dataframe descriptive statistics to for EDA
# Presenting the descriptive statistics in long format instead of wide to help 
#   with readability for the large quantity of variables
df_des = df.describe(include = 'all').T.reset_index().rename(columns = {'index':'Variable'})

# Add a column that lists the percentage of columns empty in the refined dataset
df_des['percent_empty'] = (1 - df_des['count']/len(df)) * 100

# Sort values by percent empty descending. 
df_des.sort_values(by= ['percent_empty', 'Variable'], ascending=False)

,Variable,count,unique,top,freq,mean,std,min,25%,50%,75%,max,percent_empty
155,RELAFFIL,733,57,Roman Catholic,199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.49
69,MD_EARN_WNE_INDEP1_P10,"1,600.00",NaN,NaN,NaN,"49,128.37","14,359.89","17,297.00","39,760.00","46,970.00","55,533.50","125,815.00",96.70
71,MD_EARN_WNE_INDEP1_P8,"1,629.00",NaN,NaN,NaN,"45,993.88","13,608.67","17,187.00","37,136.00","44,269.00","52,578.00","124,841.00",96.64
70,MD_EARN_WNE_INDEP1_P6,"1,665.00",NaN,NaN,NaN,"43,188.10","13,283.70","14,213.00","34,445.00","41,456.00","49,494.00","124,183.00",96.57
63,MD_EARN_WNE_INC3_P10,"1,696.00",NaN,NaN,NaN,"57,055.47","14,061.22","25,082.00","48,042.00","54,236.00","64,245.25","125,001.00",96.50
64,MD_EARN_WNE_INC3_P6,"1,724.00",NaN,NaN,NaN,"48,014.77","12,575.99","20,166.00","40,045.00","45,395.00","53,158.00","146,771.00",96.44
65,MD_EARN_WNE_INC3_P8,"1,727.00",NaN,NaN,NaN,"52,374.07","13,085.61","18,985.00","43,855.00","50,054.00","57,859.00","131,176.00",96.44
75,MD_EARN_WNE_MALE1_P10,"1,784.00",NaN,NaN,NaN,"56,604.02","16,246.22","15,475.00","47,196.50","54,361.50","64,161.25","156,181.00",96.32
60,MD_EARN_WNE_INC2_P10,"1,795.00",NaN,NaN,NaN,"51,260.70","13,365.45","19,682.00","42,984.00","49,428.00","56,270.00","127,456.00",96.30
77,MD_EARN_WNE_MALE1_P8,"1,815.00",NaN,NaN,NaN,"50,695.30","14,794.08","17,295.00","41,223.50","48,702.00","57,229.50","147,121.00",96.26


#### Export Cleaned Data

In [49]:
# write out df, df_desc, df_dictionary as cvs

# Clean up the data dictionary to include only columns in the cleaned dataset
df_dict = df_dict[df_dict['variable_name'].isin(df.columns.tolist())]

# change the column name for the following columns
df = df.rename(columns = {"STABBR":"STATE_ABBREVIATION", 'ST_FIPS': 'STATE', 
                          'CCBASIC': 'Carnegie Classification Basic', 
                          'CCSIZSET': 'Carnegie Classification size and setting',
                          'CCUGPROF': 'Carnegie Classification undergraduate profile'})


# Write out csv files
df_dict.to_csv("Reference_table.csv")
df.to_csv("w200_project2_cleaned_v1.csv")
df_des.to_csv("w200_dataset_description.csv") 

from google.colab import drive
drive.mount('./gdrive')

# Copy files to google drive folder for storage and sharing
!cp Reference_table.csv /content/gdrive/MyDrive/W200_Project_2/ # add reference table to gdrive
!cp w200_project2_cleaned_v1.csv /content/gdrive/MyDrive/W200_Project_2/ # add reference table to gdrive
!cp w200_dataset_description.csv /content/gdrive/MyDrive/W200_Project_2/ # add reference table to gdrive

Mounted at ./gdrive


# Exploratory

In [38]:
df.shape

(48477, 175)

In [39]:
df.describe(include = 'all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
ADM_RATE,"31,141.00",NaN,NaN,NaN,0.67,0.20,0.00,0.55,0.70,0.81,1.00
ADM_RATE_ALL,"31,914.00",NaN,NaN,NaN,0.67,0.20,0.00,0.55,0.69,0.81,1.00
Carnegie Classification Basic,2041,20,Master's Colleges & Universities: Larger Programs,341,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Carnegie Classification size and setting,2041,14,"Four-year, small, highly residential",372,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Carnegie Classification undergraduate profile,2041,13,"Four-year, full-time, inclusive, higher transf...",370,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CIP01BACHL,48474,3,Program not offered,43997,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CIP03BACHL,48474,3,Program not offered,34302,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CIP04BACHL,48474,3,Program not offered,43524,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CIP05BACHL,48474,3,Program not offered,35675,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CIP09BACHL,48474,3,Program offered,26035,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
df.columns.tolist()

['ADM_RATE',
 'ADM_RATE_ALL',
 'Carnegie Classification Basic',
 'Carnegie Classification size and setting',
 'Carnegie Classification undergraduate profile',
 'CIP01BACHL',
 'CIP03BACHL',
 'CIP04BACHL',
 'CIP05BACHL',
 'CIP09BACHL',
 'CIP10BACHL',
 'CIP11BACHL',
 'CIP12BACHL',
 'CIP13BACHL',
 'CIP14BACHL',
 'CIP15BACHL',
 'CIP16BACHL',
 'CIP19BACHL',
 'CIP22BACHL',
 'CIP23BACHL',
 'CIP24BACHL',
 'CIP25BACHL',
 'CIP26BACHL',
 'CIP27BACHL',
 'CIP29BACHL',
 'CIP30BACHL',
 'CIP31BACHL',
 'CIP38BACHL',
 'CIP39BACHL',
 'CIP40BACHL',
 'CIP41BACHL',
 'CIP42BACHL',
 'CIP43BACHL',
 'CIP44BACHL',
 'CIP45BACHL',
 'CIP46BACHL',
 'CIP47BACHL',
 'CIP48BACHL',
 'CIP49BACHL',
 'CIP50BACHL',
 'CIP51BACHL',
 'CIP52BACHL',
 'CIP54BACHL',
 'CITY',
 'CONTROL',
 'COSTT4_A',
 'ENDYEAR',
 'HBCU',
 'HIGHDEG',
 'HSI',
 'INSTNM',
 'MAIN',
 'MDCOMP_PD',
 'MDCOST_ALL',
 'MDCOST_PD',
 'MDEARN_ALL',
 'MDEARN_PD',
 'MD_EARN_WNE_INC1_P10',
 'MD_EARN_WNE_INC1_P6',
 'MD_EARN_WNE_INC1_P8',
 'MD_EARN_WNE_INC2_P10',
 'MD_E

In [41]:
# Reference dataframe that has the column shorthand with the description
df_dict.head()

,name_of_data_element,dev_category,developer_friendly_name,api_data_type,index,variable_name,value,label,source,shown_use_on_site,notes
0,Unit ID for institution,root,id,integer,NaN,UNITID,NaN,NaN,IPEDS,Yes,NaN
3,Institution name,school,name,autocomplete,fulltext,INSTNM,NaN,NaN,IPEDS,Yes,NaN
4,City,school,city,autocomplete,varchar(200),CITY,NaN,NaN,IPEDS,Yes,NaN
5,State postcode,school,state,string,varchar(50),STABBR,NaN,NaN,IPEDS,Yes,NaN
6,ZIP code,school,zip,string,varchar(20),ZIP,NaN,NaN,IPEDS,NaN,NaN


## Exploration

In [42]:
# create correlation matrix to look at correlated columns
data_corr = df.corr()
data_tril = data_corr.where(~np.tril(np.ones(data_corr.shape)).astype(np.bool))
data_tril = data_tril.stack()
# cors = data_tril[(data_tril > 0.7) | (data_tril < -0.7)] # Use to restrict to correlations greater then an absolute value of correlation greater than 0.7


`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [43]:
# Look at the columns with correlations
df_corr = pd.DataFrame(data_tril)
df_corr.reset_index(inplace=True)
df_corr.rename(columns={'level_0': 'var1', 'level_1': 'var2'}, inplace=True) # Rename with to easiery column names
df_corr = df_corr.reindex(columns= df_corr.columns.to_list())
df_corr = df_corr.rename(columns={0:'Correlation'})

In [44]:
# Add descriptions to correlation to make it easier to understand
corr_dict = dict(zip(df_dict['variable_name'], df_dict['name_of_data_element']))
df_corr['var1_description'] = df_corr['var1'].map(corr_dict)
df_corr['var2_description'] = df_corr['var2'].map(corr_dict)


In [45]:
df_corr.sort_values(by = 'Correlation', ascending = False).head(10)

,var1,var2,Correlation,var1_description,var2_description
2410,NPT41_PUB,NPT4_048_PUB,0.99,"Average net price for $0-$30,000 family income...","Average net price for $0-$48,000 family income..."
2349,NPT41_PRIV,NPT4_048_PRIV,0.99,"Average net price for $0-$30,000 family income...","Average net price for $0-$48,000 family income..."
1186,MD_EARN_WNE_INDEP0_P10,MD_EARN_WNE_P10,0.99,Median earnings of dependent students working ...,Median earnings of students working and not en...
1355,MD_EARN_WNE_INDEP0_P8,MD_EARN_WNE_P8,0.98,Median earnings of dependent students working ...,Median earnings of students working and not en...
2590,NPT43_PRIV,NPT4_3075_PRIV,0.98,"Average net price for $48,001-$75,000 family i...","Average net price for $30,001-$75,000 family i..."
2880,NPT45_PUB,NPT4_75UP_PUB,0.98,"Average net price for $110,000+ family income ...","Average net price for $75,000+ family income (..."
1271,MD_EARN_WNE_INDEP0_P6,MD_EARN_WNE_P6,0.98,Median earnings of dependent students working ...,Median earnings of students working and not en...
2766,NPT44_PUB,NPT4_75UP_PUB,0.98,"Average net price for $75,001-$110,000 family ...","Average net price for $75,000+ family income (..."
2823,NPT45_PRIV,NPT4_75UP_PRIV,0.98,"Average net price for $110,000+ family income ...","Average net price for $75,000+ family income (..."
2649,NPT43_PUB,NPT4_3075_PUB,0.98,"Average net price for $48,001-$75,000 family i...","Average net price for $30,001-$75,000 family i..."


In [46]:
# First 10 columns from the data correlation matrix
df_corr.head(10)

,var1,var2,Correlation,var1_description,var2_description
0,ADM_RATE,ADM_RATE_ALL,0.98,Admission rate,Admission rate for all campuses rolled up to t...
1,ADM_RATE,COSTT4_A,-0.27,Admission rate,Average cost of attendance (academic year inst...
2,ADM_RATE,ENDYEAR,-0.05,Admission rate,NaN
3,ADM_RATE,MD_EARN_WNE_INC1_P10,-0.28,Admission rate,Median earnings of students working and not en...
4,ADM_RATE,MD_EARN_WNE_INC1_P6,-0.21,Admission rate,Median earnings of students working and not en...
5,ADM_RATE,MD_EARN_WNE_INC1_P8,-0.25,Admission rate,Median earnings of students working and not en...
6,ADM_RATE,MD_EARN_WNE_INC2_P10,-0.24,Admission rate,Median earnings of students working and not en...
7,ADM_RATE,MD_EARN_WNE_INC2_P6,-0.18,Admission rate,Median earnings of students working and not en...
8,ADM_RATE,MD_EARN_WNE_INC2_P8,-0.22,Admission rate,Median earnings of students working and not en...
9,ADM_RATE,MD_EARN_WNE_INC3_P10,-0.25,Admission rate,Median earnings of students working and not en...
